# Analysis of Olist reviews 

Analyse the Olist reviews to understand what could be the causes of the bad review scores.

In [1]:
# import modules 

import pandas as pd
import numpy as np 
import string
import unidecode

from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords 

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.pipeline import Pipeline

from googletrans import Translator

from olistpackage.data import Olist

In [2]:
# magic commands

%load_ext autoreload
%autoreload 2

In [3]:
# Instantiate the Olist class 
# which main purpose is downloading Olist csv files.

olist = Olist()

In [4]:
# Download Olist csv files in root/data/csv folder.
# If all the csv files exists already, do nothing.
# Uses the Kaggle Official API for donwloading Olist csv files.
# Kaggle Official API credentials are asked by opendatasets.download().
# Sign in to https://kaggle.com/, then click on your profile picture
# on the top right and select "My Account" from the menu.
# Scroll down to the "API" section and click "Create New API Token".
# This will download a file kaggle.json with the following contents:
# {"username":"YOUR_KAGGLE_USERNAME","key":"YOUR_KAGGLE_KEY"}

olist.download_data()

The Olist csv files are already downloaded.


In [5]:
# get the Olist csv files data into a dictionary of dataframes
data = olist.get_data()

In [6]:
# get the dataframe on which the sentiment analysis will be done
df = data['order_reviews'].merge(data['orders'], on="order_id", how="inner")

In [7]:
df.shape

(99224, 14)

In [8]:
df.columns

Index(['review_id', 'order_id', 'review_score', 'review_comment_title',
       'review_comment_message', 'review_creation_date',
       'review_answer_timestamp', 'customer_id', 'order_status',
       'order_purchase_timestamp', 'order_approved_at',
       'order_delivered_carrier_date', 'order_delivered_customer_date',
       'order_estimated_delivery_date'],
      dtype='object')

In [9]:
# combine review title and review message

df["review_title_and_message"] = df["review_comment_title"].fillna("") \
    + " " + df["review_comment_message"].fillna("")

In [10]:
# Customers could review an order before receiving it
# Let's consider reviews written only after receiving the order 

df = df.query("review_creation_date >= order_delivered_customer_date")
df.shape

(88039, 15)

In [11]:
# check the stop words list before cleaning the reviews
list(stopwords.words("portuguese"))

['de',
 'a',
 'o',
 'que',
 'e',
 'é',
 'do',
 'da',
 'em',
 'um',
 'para',
 'com',
 'não',
 'uma',
 'os',
 'no',
 'se',
 'na',
 'por',
 'mais',
 'as',
 'dos',
 'como',
 'mas',
 'ao',
 'ele',
 'das',
 'à',
 'seu',
 'sua',
 'ou',
 'quando',
 'muito',
 'nos',
 'já',
 'eu',
 'também',
 'só',
 'pelo',
 'pela',
 'até',
 'isso',
 'ela',
 'entre',
 'depois',
 'sem',
 'mesmo',
 'aos',
 'seus',
 'quem',
 'nas',
 'me',
 'esse',
 'eles',
 'você',
 'essa',
 'num',
 'nem',
 'suas',
 'meu',
 'às',
 'minha',
 'numa',
 'pelos',
 'elas',
 'qual',
 'nós',
 'lhe',
 'deles',
 'essas',
 'esses',
 'pelas',
 'este',
 'dele',
 'tu',
 'te',
 'vocês',
 'vos',
 'lhes',
 'meus',
 'minhas',
 'teu',
 'tua',
 'teus',
 'tuas',
 'nosso',
 'nossa',
 'nossos',
 'nossas',
 'dela',
 'delas',
 'esta',
 'estes',
 'estas',
 'aquele',
 'aquela',
 'aqueles',
 'aquelas',
 'isto',
 'aquilo',
 'estou',
 'está',
 'estamos',
 'estão',
 'estive',
 'esteve',
 'estivemos',
 'estiveram',
 'estava',
 'estávamos',
 'estavam',
 'estivera'

In [12]:
# Cleaning the reviews

def remove_punctuation(text=""):
    return "".join([char for char in text if char not in string.punctuation])

def lower_case(text=""):
    return text.lower()

def remove_number(text=""):
    return "".join([char for char in text if not char.isdigit()])

def remove_accents_diacritics(text=""):
    return unidecode.unidecode(text)

def tokenize(text=""):
    return word_tokenize(text)
    

def stop_words(text=""):
    return " ".join([word for word in text if word not in stopwords.words("portuguese")])

def strip_white_spaces(text=""):
    return text.strip()

funcs = [
    remove_punctuation,
    lower_case,
    remove_number,
    remove_accents_diacritics,
    tokenize,
    stop_words,
    strip_white_spaces
]

def clean(text=""):
    for f in funcs:
        text = f(text)
    return text

In [13]:
# Create a new column for cleaned reviews and call clean function on it

df["clean_review"] = df["review_title_and_message"].apply(clean)
df.shape

(88039, 16)

In [14]:
# drop the empty reviews

df = df[df["clean_review"] != ""]
df.shape

(35983, 16)

The dataset contains more than 35000 cleaned reviews.

In [15]:
# Check the unique values of reviews score

np.sort(df['review_score'].unique())

array([1, 2, 3, 4, 5])

In [16]:
# review score distribution

round(df["review_score"].value_counts(normalize = True), 2)

5    0.57
4    0.16
1    0.13
3    0.09
2    0.05
Name: review_score, dtype: float64

More than 25% of the orders with a review have a score equal or below to 3.
Let's focus on those orders' reviews.

In [17]:
df = df[df["review_score"]<=3]
df.shape

(9599, 16)

The sentiment analysis will be done on 9599 cleaned reviews.

In [18]:
# Create Pipeline with TfidfVectorizer and LatentDirichletAllocation
# to vizualize potential topics

pipeline = Pipeline([
    ('TfidfVectorizer', TfidfVectorizer(ngram_range=(2, 2))),
    ('LatentDirichletAllocation', LatentDirichletAllocation(n_components=2))
])

In [19]:
pipeline.fit(df["clean_review"])

Pipeline(steps=[('TfidfVectorizer', TfidfVectorizer(ngram_range=(2, 2))),
                ('LatentDirichletAllocation',
                 LatentDirichletAllocation(n_components=2))])

In [20]:
vectorizer, model = pipeline[0], pipeline[1]

In [21]:
# get the main topics of the reviews
# use Google translator to translate from portuguese to english

translator = Translator()

for idx, topic in enumerate(model.components_):
    print("Topic %d:" % (idx))
    
    print([(translator.translate(vectorizer.get_feature_names_out()[i], src="pt", dest="en").text, topic[i])
           for i in topic.argsort()[:-10 - 1:-1]])

Topic 0:
[('not received', 144.99383191066187), ('I received product', 115.54227650255888), ('not yet', 79.32889929301336), ('no product', 69.96027666046231), ('not delivered', 49.30835158100186), ('Product came', 45.636167456954325), ('wrong product', 38.37825447884039), ('product delivered', 37.09956597156733), ('did not like', 33.89317683974378), ('did not come', 33.220517903312)]
Topic 1:
[('good product', 37.66935324657181), ('good product', 36.2503703183614), ('Product came', 34.26766163799318), ('no product', 30.03986268903552), ('defective product', 29.68230910309153), ('SO I RECEIVED', 27.92961357035397), ('I bought two', 27.744310376619012), ('I do not recommend', 27.6323224928698), ('I liked product', 24.74075771809443), ('two products', 24.656978825400095)]


It seems that the bad reviews are caused by:   
1 - delivery delay   
2 - problem with the product itself (wrong one, wrong quantity, defective)   

Let's make a deeper analysis

In [22]:
# new vectorizer, with TfidfVectorizer

vectorizer2 = TfidfVectorizer(ngram_range = (2, 2),
                             min_df=0.01,
                             max_df=0.05).fit(df["clean_review"])

In [23]:
# transform the cleaned reviews into vectors
vectors = pd.DataFrame(vectorizer.transform(df["clean_review"]).toarray(),
                       columns = vectorizer.get_feature_names_out())
vectors.shape

(9599, 53180)

In [24]:
# sum the components of each vectors
sum_tfidf = vectors.sum(axis = 0)
sum_tfidf.shape

(53180,)

In [25]:
# list of words and weight
tfidf_list = [(word, sum_tfidf[word]) 
              for word, idx in vectorizer.vocabulary_.items() 
              if word in vectorizer.vocabulary_.keys() ]
len(tfidf_list)

53180

In [29]:
# sort the tfidf_list
sorted_tfidf_list = sorted(tfidf_list, key = lambda x: x[1], reverse=True)
sorted_tfidf_list[:10]

[('nao recebi', 153.44975401815063),
 ('recebi produto', 120.38649776008641),
 ('produto nao', 99.00013934951842),
 ('ainda nao', 81.53229412149304),
 ('produto veio', 78.90382909496803),
 ('nao entregue', 56.434617957759606),
 ('nao gostei', 53.42165795526988),
 ('so recebi', 53.401861134327255),
 ('nao recomendo', 52.51305502454542),
 ('produto entregue', 50.189472437204266)]

In [30]:
# use Google translator to translate from portuguese to english and round the weights

translator = Translator()
translated_sorted_tfidf_list = [
    (translator.translate(t, src="pt", dest="en").text, round(w, 2)) for t, w in sorted_tfidf_list[:50]]

translated_sorted_tfidf_list

[('not received', 153.45),
 ('I received product', 120.39),
 ('no product', 99.0),
 ('not yet', 81.53),
 ('Product came', 78.9),
 ('not delivered', 56.43),
 ('did not like', 53.42),
 ('SO I RECEIVED', 53.4),
 ('I do not recommend', 52.51),
 ('product delivered', 50.19),
 ('I received only', 46.8),
 ('Product arrived', 45.59),
 ('did not come', 43.98),
 ('did not arrive', 41.52),
 ('invoice', 39.39),
 ('wrong product', 39.12),
 ('good product', 38.19),
 ('Come defect', 36.46),
 ('good product', 35.94),
 ('I bought two', 35.84),
 ('does not work', 33.56),
 ('yet', 33.38),
 ('rather', 32.85),
 ('pessimal quality', 31.15),
 ('defective product', 29.85),
 ('within', 29.62),
 ('so far', 29.31),
 ('I liked product', 28.69),
 ('I bought product', 27.89),
 ('deadline', 27.4),
 ('Wrong came', 26.38),
 ('came missing', 25.13),
 ('two products', 24.77),
 ('just came', 23.41),
 ('product bought', 23.23),
 ('I bought two', 23.03),
 ('product yet', 22.17),
 ('I received only', 21.0),
 ('just arrived'

This confirm the previous analysis:

1 - delivery issues: delay and no delivery   
2 - problem with the product itself (bad quality, defective product, wrong product, wrong quantity)   